In [1]:
import warnings
warnings.simplefilter('ignore')

import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

# Modelo InceptionV3

In [2]:
from keras.applications import inception_v3
from keras import backend as K

K.set_learning_phase(0)

Using TensorFlow backend.


In [3]:
# solo la parte convolucional

model = inception_v3.InceptionV3(weights='imagenet',
                                 include_top=False)

**configuracion**

In [4]:
# cuanto contribuye cada capa a la maximizacion

layer_contributions = {
    'mixed2': 0.2,
    'mixed3': 3.,
    'mixed4': 2.,
    'mixed5': 1.5,
}

**funcion de perdida a maximizar**

In [5]:
layer_dict = dict([(layer.name, layer) for layer in model.layers])


loss = K.variable(0.) # perdida

for layer_name in layer_contributions:
    coeff = layer_contributions[layer_name]
    activation = layer_dict[layer_name].output  # salida de la capa
    
    scaling = K.prod(K.cast(K.shape(activation), 'float32')) # norma L2 
    
    loss += coeff * K.sum(K.square(activation[:, 2: -2, 
                                              2: -2, :])) / scaling

**proceso de gradiente ascendente**

In [6]:
dream = model.input  # se genera la imagen

grads = K.gradients(loss, dream)[0]  # gradientes
grads /= K.maximum(K.mean(K.abs(grads)), 1e-7) # normalizacion

# se obtienen los valores de perdida y su gradiente
outputs = [loss, grads]
fetch_loss_and_grads = K.function([dream], outputs) 


# se evalua
def eval_loss_and_grads(x):
    outs = fetch_loss_and_grads([x])
    loss_value = outs[0]
    grad_values = outs[1]
    return loss_value, grad_values



# gradiente ascente en n iteraciones
def gradient_ascent(x, iterations, step, max_loss=None):
    
    for i in range(iterations):
        loss_value, grad_values = eval_loss_and_grads(x)
        if max_loss is not None and loss_value > max_loss:
            break
        
        print('...Loss value at', i, ':', loss_value)
        x += step * grad_values
        
    return x

**funciones auxiliares**

In [7]:
import scipy
import numpy as np
import imageio
from keras.preprocessing import image

def resize_img(img, size):
    img = np.copy(img)
    factors = (1,
               float(size[0]) / img.shape[1],
               float(size[1]) / img.shape[2],
               1)
    return scipy.ndimage.zoom(img, factors, order=1)

In [8]:
def save_img(img, fname):
    pil_img = deprocess_image(np.copy(img))
    imageio.imwrite(fname, pil_img)

In [9]:
def preprocess_image(image_path):
    img = image.load_img(image_path)
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = inception_v3.preprocess_input(img)
    return img

In [10]:
def deprocess_image(x):
    if K.image_data_format() == 'channels_first':
        x = x.reshape((3, x.shape[2], x.shape[3]))
        x = x.transpose((1, 2, 0))
    else:
        x = x.reshape((x.shape[1], x.shape[2], 3))
    x /= 2.
    x += 0.5
    x *= 255.
    x = np.clip(x, 0, 255).astype('uint8')
    return x

**sucesivas escalas (octavas)**

In [11]:
step = 0.01

num_octave = 5
octave_scale = 1.4

iterations = 20

max_loss = 10.


base_image_path = 'data/dream.jpg'
img = preprocess_image(base_image_path)

In [12]:
original_shape = img.shape[1:3]
successive_shapes = [original_shape]

for i in range(1, num_octave):
    shape = tuple([int(dim / (octave_scale ** i))
        for dim in original_shape])
    successive_shapes.append(shape)
    
successive_shapes = successive_shapes[::-1]

original_img = np.copy(img)
shrunk_original_img = resize_img(img, successive_shapes[0])

In [13]:
for shape in successive_shapes:
    print('Processing image shape', shape)
    img = resize_img(img, shape)
    img = gradient_ascent(img,
                          iterations=iterations,
                          step=step,
                          max_loss=max_loss)
    
    upscaled_shrunk_original_img = resize_img(shrunk_original_img, 
                                              shape) 
    
    same_size_original = resize_img(original_img, shape)
    
    lost_detail = same_size_original - upscaled_shrunk_original_img
    img += lost_detail
    
    shrunk_original_img = resize_img(original_img, shape)
    #save_img(img, fname='dream_at_scale_' + str(shape) + '.png')

Processing image shape (58, 58)
...Loss value at 0 : 0.0
...Loss value at 1 : 0.0
...Loss value at 2 : 0.0
...Loss value at 3 : 0.0
...Loss value at 4 : 0.0
...Loss value at 5 : 0.0
...Loss value at 6 : 0.0
...Loss value at 7 : 0.0
...Loss value at 8 : 0.0
...Loss value at 9 : 0.0
...Loss value at 10 : 0.0
...Loss value at 11 : 0.0
...Loss value at 12 : 0.0
...Loss value at 13 : 0.0
...Loss value at 14 : 0.0
...Loss value at 15 : 0.0
...Loss value at 16 : 0.0
...Loss value at 17 : 0.0
...Loss value at 18 : 0.0
...Loss value at 19 : 0.0
Processing image shape (81, 81)
...Loss value at 0 : 0.026156599
...Loss value at 1 : 0.035130437
...Loss value at 2 : 0.043979026
...Loss value at 3 : 0.052219845
...Loss value at 4 : 0.060756657
...Loss value at 5 : 0.06980564
...Loss value at 6 : 0.07970229
...Loss value at 7 : 0.089084744
...Loss value at 8 : 0.095858164
...Loss value at 9 : 0.10423756
...Loss value at 10 : 0.11375146
...Loss value at 11 : 0.11671465
...Loss value at 12 : 0.12503392


In [14]:
save_img(img, fname='data/final_dream.png')